Reads in a geopackage and returns a metadata csv

In [13]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import box

In [2]:
# relative paths to the mra geopackage files 
egpkg = 'mra/England/Public Sector England/England_GPKG/England_GPKG.gpkg'
sgpkg = 'mra/Scotland/Public Sector Scotland/Scotland_GPKG/Scotland_GPKG.gpkg'
wgpkg = 'Wales/Public Sector Wales/Wales_GPKG/Wales_GPKG.gpkg'

In [ ]:
# retrieves a list of layers in the geopackage as layer_list
pkg_layers = gpd.list_layers(egpkg)
layer_list = pkg_layers['name'].values.tolist()
layer_list

In [4]:
# Creates a pandas dataframe with the boundary, geometry, and geometry type as columns 
lbounds = list()
lgeom = list()
lgt = list()
gtlist = list()
for i in layer_list: 
    lay = gpd.read_file(egpkg, layer=i)
    layer_bounds = lay.geometry.total_bounds
    lbounds.append(layer_bounds)
    geom = box(*lay.geometry.total_bounds)
    lgeom.append(geom)
    gt = lay.geom_type.unique()
    gtlist.append(gt)
lpd = pd.DataFrame({'name':layer_list,'bounds':lbounds,'geometry':lgeom,'geomtype':gtlist})
lpd.head()

C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCach

,name,bounds,geometry,geomtype
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]"
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99000000022, 633740 65...",[Point]
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.75999999978, 489617 65...","[Polygon, None]"
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString]
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38000000082, 453883 65...",[LineString]


In [5]:
# converts the dataframe to a geopandas datarame and adds a column with the geometry in epsg4326 (which can be plotted in folium)
lgpd = gpd.GeoDataFrame(lpd, crs="EPSG:27700")
lgpd['epsg4326'] = lgpd['geometry'].to_crs("EPSG:4326")
lgpd.head()

,name,bounds,geometry,geomtype,epsg4326
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]","POLYGON ((1.55745 50.9432, 1.98283 55.75521, -..."
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99, 633740 658335.54, ...",[Point],"POLYGON ((1.32843 50.97827, 1.72418 55.76185, ..."
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.76, 489617 651236, 244...","[Polygon, None]","POLYGON ((-0.72368 51.0204, -0.57399 55.74622,..."
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString],"POLYGON ((-0.95617 51.21754, -0.83815 55.76483..."
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38, 453883 652643, 245...",[LineString],"POLYGON ((-1.23319 51.01824, -1.14285 55.76414..."


In [ ]:
# gives geodata details of the dataframe
lgpd.crs

In [6]:
lbounds

[array([240000., 122000., 650000., 658500.]),
 array([245082.87  , 125147.99  , 633740.    , 658335.5395]),
 array([244983.42, 125333.76, 489617.  , 651236.  ]),
 array([296168., 147000., 473000., 652998.]),
 array([245408.7 , 124597.38, 453883.  , 652643.  ]),
 array([297155., 145979., 637738., 647834.]),
 array([305027.    , 299539.31  , 466462.2   , 582291.9284]),
 array([280001.31, 134709.  , 643224.7 , 642873.  ]),
 array([246967.82, 126117.5 , 637099.  , 651584.31]),
 array([294825., 147363., 637127., 648541.]),
 array([246966.97, 126116.63, 636930.  , 651592.44]),
 array([246967.83, 126117.4 , 637099.  , 651584.34]),
 array([298734.    , 178055.6561, 452227.    , 641601.    ]),
 array([246912.69, 126165.8 , 636902.  , 650785.  ]),
 array([247043.53, 126121.32, 637042.  , 650878.14]),
 array([298134.39, 209848.71, 462545.03, 556483.66]),
 array([382379., 295145., 469373., 424003.]),
 array([383126., 341816., 455193., 370078.]),
 array([nan, nan, nan, nan]),
 array([nan, nan, nan,

In [ ]:
ops = gpd.read_file(egpkg, layer='Coalfield_Consultation_Area')
ops_bounds = ops.geometry.total_bounds
ops_bounds

array([240000., 122000., 650000., 658500.])

In [12]:
print(type(ops_bounds))

<class 'numpy.ndarray'>


In [17]:
wb = ops_bounds[0]
wb

np.float64(240000.0)

In [19]:
wbb = formatCoord(ops_bounds,'WE')
wbb

[240000. 122000. 650000. 658500.] WE E240000.0


'E240000.0122000.0650000.0'

In [ ]:
w1 = 'E'+str(int(ops_bounds[0]))
print(type(w1),w1)

TypeError: can only concatenate str (not "int") to str

In [25]:
# w = int(ops_bounds[0])
# s = int(ops_bounds[0])
# e = int(ops_bounds[0])

np.isnan(ops_bounds[0])

np.False_

In [47]:
# clgpd['Bounding_Box_W'] = 'E'+str(int(clgpd['bounds']))

# create a dataframe of values for each layer in the geopackage with bounds, geometry, geometry type and bounding box values
lbounds, lgeom, lgt, gtlist, bbw, bbs, bbe, bbn  = list(), list(), list(), list(), list(), list(), list(), list()

for i in layer_list:
    lay = gpd.read_file(egpkg, layer=i)
    layer_bounds = lay.geometry.total_bounds
    wval = getbbe(layer_bounds[0],'WE')
    sval = getbbe(layer_bounds[1],'NS')
    eval = getbbe(layer_bounds[2],'WE')
    nval = getbbe(layer_bounds[3],'NS')
    bbw.append(wval)
    bbs.append(sval)
    bbe.append(eval)
    bbn.append(nval)
    lbounds.append(layer_bounds)
    geom = box(*lay.geometry.total_bounds)
    lgeom.append(geom)
    gt = lay.geom_type.unique()
    gtlist.append(gt)
tlpd = pd.DataFrame({'name':layer_list,'bounds':lbounds,'geometry':lgeom,'geomtype':gtlist, 'Bounding_Box_W':bbw, 'Bounding_Box_E':bbe, 'Bounding_Box_N':bbn, 'Bounding_Box_S':bbs})
tlpd.head()

C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCach

,name,bounds,geometry,geomtype,Bounding_Box_W,Bounding_Box_E,Bounding_Box_N,Bounding_Box_S
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]",E240000,E650000,N658500,N122000
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99000000022, 633740 65...",[Point],E245082,E633740,N658335,N125147
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.75999999978, 489617 65...","[Polygon, None]",E244983,E489617,N651236,N125333
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString],E296168,E473000,N652998,N147000
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38000000082, 453883 65...",[LineString],E245408,E453883,N652643,N124597


In [45]:
def getbbe(bvalue,dir):
    if np.isnan(bvalue):
        res = None
    else:
        bvint = int(bvalue)
        if bvint >= 0:
            if dir == 'WE':
                res = 'E'+str(bvint)
            else:
                res = 'N'+str(bvint)
        else:
            if dir == 'WE':
                res = 'W'+str(bvint)
            else:
                res = 'S'+str(bvint)
    return res    

In [ ]:

# create a dataframe of values for each layer in the geopackage with bounds, geometry, geometry type and bounding box values
lbounds, lgeom, lgt, gtlist, bbw, bbs, bbe, bbn  = list(), list(), list(), list(), list(), list(), list(), list()

for i in layer_list:
    lay = gpd.read_file(egpkg, layer=i)
    layer_bounds = lay.geometry.total_bounds
    w1 = getbwe(layer_bounds[0])
    lbw.append(w1)
    lbounds.append(layer_bounds)
    geom = box(*lay.geometry.total_bounds)
    lgeom.append(geom)
    gt = lay.geom_type.unique()
    gtlist.append(gt)
tlpd = pd.DataFrame({'name':layer_list,'bounds':lbounds,'geometry':lgeom,'geomtype':gtlist, 'Bounding_Box_W':lbw})
tlpd.head()

In [44]:
tlpd

,name,bounds,geometry,geomtype,bbw
0,tl,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]",E240000
1,tl,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99000000022, 633740 65...",[Point],E245082
2,tl,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.75999999978, 489617 65...","[Polygon, None]",E244983
3,tl,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString],E296168
4,tl,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38000000082, 453883 65...",[LineString],E245408
5,tl,"[297155.0, 145979.0, 637738.0, 647834.0]","POLYGON ((637738 145979, 637738 647834, 297155...",[Polygon],E297155
6,tl,"[305027.0, 299539.3100000005, 466462.200000000...","POLYGON ((466462.2000000002 299539.3100000005,...",[Polygon],E305027
7,tl,"[280001.3099999996, 134709.0, 643224.700000000...","POLYGON ((643224.7000000002 134709, 643224.700...",[Polygon],E280001
8,tl,"[246967.8200000003, 126117.5, 637099.0, 651584...","POLYGON ((637099 126117.5, 637099 651584.31000...",[Polygon],E246967
9,tl,"[294825.0, 147363.0, 637127.0, 648541.0]","POLYGON ((637127 147363, 637127 648541, 294825...",[LineString],E294825


In [ ]:
def getbbe(b):
    if np.isnan(b):
        res = None
    else:
        bin = int(b)
        if bin > 0:
            print('pos',bin)
            res = 'E'+str(bin)
        else:
            print('neg',bin)
            res = 'W'+str(bin)
    return res    
           

In [31]:
lpd

,name,bounds,geometry,geomtype
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]"
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99000000022, 633740 65...",[Point]
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.75999999978, 489617 65...","[Polygon, None]"
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString]
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38000000082, 453883 65...",[LineString]
5,Legal_Notice,"[297155.0, 145979.0, 637738.0, 647834.0]","POLYGON ((637738 145979, 637738 647834, 297155...",[Polygon]
6,Licence_Area_of_Responsibility,"[305027.0, 299539.3100000005, 466462.200000000...","POLYGON ((466462.2000000002 299539.3100000005,...",[Polygon]
7,Licence_Area,"[280001.3099999996, 134709.0, 643224.700000000...","POLYGON ((643224.7000000002 134709, 643224.700...",[Polygon]
8,Parent_Phase,"[246967.8200000003, 126117.5, 637099.0, 651584...","POLYGON ((637099 126117.5, 637099 651584.31000...",[Polygon]
9,In_Seam_Level_Contour,"[294825.0, 147363.0, 637127.0, 648541.0]","POLYGON ((637127 147363, 637127 648541, 294825...",[LineString]


In [16]:
def formatCoord (bounds, direction):
    threeDigitBounds = '{:03}'.format(bounds[0]).replace('-', '0')
    if bounds[0] >= 0:
        if direction == 'WE':
            threeDigitBounds = 'E' + threeDigitBounds
        elif direction == 'NS':
            threeDigitBounds = 'N' + threeDigitBounds
    elif int(bounds[0]) < 0:
        if direction == 'WE':
            threeDigitBounds = 'W' + threeDigitBounds
        elif direction == 'NS':
            threeDigitBounds = 'S' + threeDigitBounds
    print(bounds, direction,threeDigitBounds)
    return threeDigitBounds + '{:02}'.format(bounds[1]) + '{:02}'.format(bounds[2])

In [21]:
# removes any layers with invalid geometry
clgpd = lgpd.loc[lgpd['geometry'].is_valid, :]

In [19]:
# splits the geometry into columns and appends to the table 
bounding_boxes = lgpd.geometry.bounds
layergpd = lgpd.join(bounding_boxes)
layergpd.head()

,name,bounds,geometry,geomtype,epsg4326,minx,miny,maxx,maxy
0,Coalfield_Consultation_Area,"[184000.0, 179000.0, 369000.0, 409073.8800000008]","POLYGON ((369000 179000, 369000 409073.88, 184...",[Polygon],"POLYGON ((-2.44807 51.50907, -2.46965 53.57734...",184000.00,179000.000,369000.00,409073.88
2,Mine_Entry,"[185045.09999999963, 180257.3719999995, 367012...","POLYGON ((367012.42 180257.372, 367012.42 3837...",[Point],"POLYGON ((-2.47683 51.52026, -2.497 53.34958, ...",185045.10,180257.372,367012.42,383747.80
3,Parent_Phase,"[185532.0, 180691.0, 366643.0, 385650.0]","POLYGON ((366643 180691, 366643 385650, 185532...",[Polygon],"POLYGON ((-2.48219 51.52414, -2.50275 53.36666...",185532.00,180691.000,366643.00,385650.00
4,Probable_Workings,"[186271.0, 182557.0, 365908.0, 383259.0]","POLYGON ((365908 182557, 365908 383259, 186271...","[Polygon, None]","POLYGON ((-2.49297 51.54087, -2.51354 53.34512...",186271.00,182557.000,365908.00,383259.00
5,In_Seam_Level,"[185561.63999999966, 180708.0, 365881.59999999...","POLYGON ((365881.6 180708, 365881.6 385389, 18...",[Point],"POLYGON ((-2.49317 51.52425, -2.51417 53.36426...",185561.64,180708.000,365881.60,385389.00


In [20]:
# outputs to csv 
layergpd.to_csv('gpkg_meta.csv')